In [130]:
import pandas as pd
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from imblearn.over_sampling import RandomOverSampler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

data_file = "dat.csv"

df = pd.read_csv(data_file)

cols = ["day_of_week", "hour_of_day", "motion_duration"]
X = df[cols].values
y = df["label"].values

In [131]:
class EarlyStopping:
    def __init__(self, patience=4, delta=0.0):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_state = None

    def __call__(self, val_loss, model):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.best_state = model.state_dict()
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
scaler = MinMaxScaler()

df[cols] = scaler.fit_transform(df[cols]) # to normalize values since some are orders of magnitude greater than others, e.g. motion_duration

print(df)

SEQ_LEN = 10

X_seq, Y_seq = [], []

# sliding window to make preds on each timestep
for i in range(len(X) - SEQ_LEN):
    X_seq.append(X[i:i+SEQ_LEN])
    Y_seq.append(y[i+SEQ_LEN])



X_train, X_test, Y_train, Y_test = train_test_split(X_seq, Y_seq, test_size=0.2, random_state=32)

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.int64)
Y_test = torch.tensor(Y_test, dtype=torch.int64)
class_counts = np.bincount(Y_train)
class_weights = 1.0 / class_counts
sample_weights = class_weights[Y_train]

print(f"X_train shape: {X_train.shape}")
print(f"Y_train shape: {Y_train.shape}")

from torch.utils.data import WeightedRandomSampler

class_counts = np.bincount(Y_train)
class_weights = 1.0 / class_counts
sample_weights = class_weights[Y_train]

print("sample weights", sample_weights)

print(Counter(all_labels))

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(TensorDataset(X_train,
                                        Y_train),
                          batch_size=32, sampler=sampler)
test_loader = DataLoader(TensorDataset(X_test, Y_test), batch_size=32)


from collections import Counter

all_labels = []

for _, labels in train_loader:
    all_labels.extend(labels.numpy().astype(int))

print(Counter(all_labels))

               timestamp  day_of_week  hour_of_day  motion_duration  label
0    2024-05-20 02:15:00          0.0     0.086957         0.749948      1
1    2024-05-20 07:15:00          0.0     0.304348         0.017295      0
2    2024-05-20 12:30:00          0.0     0.521739         0.033132      0
3    2024-05-20 14:00:00          0.0     0.608696         0.001875      0
4    2024-05-20 18:45:00          0.0     0.782609         0.062305      0
..                   ...          ...          ...              ...    ...
703  2024-09-15 03:15:00          1.0     0.130435         0.266514      0
704  2024-09-15 07:35:00          1.0     0.304348         0.070640      1
705  2024-09-15 11:55:00          1.0     0.478261         0.053136      1
706  2024-09-15 18:35:00          1.0     0.782609         0.074807      0
707  2024-09-15 23:00:00          1.0     1.000000         0.062305      0

[708 rows x 5 columns]
X_train shape: torch.Size([558, 10, 3])
Y_train shape: torch.Size([558])
sam

In [133]:
class Detector(nn.Module):
    def __init__(self, input_size=3, hidden_size=32, num_layers=2, bidirectional=True):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, bidirectional=bidirectional) # perhaps make bidirectional to learn patterns relating to other motions
        lstm_output_size = hidden_size * 2 if bidirectional else hidden_size
        self.fc = nn.Linear(lstm_output_size, 1)
        print(f"creating Detector with hidden_size={hidden_size}, and num_layers={num_layers}")

    def forward(self, x):
        out, _ = self.lstm(x)
        return torch.sigmoid(self.fc(out[:, -1, :])).squeeze()


In [154]:
model = Detector()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

creating Detector with hidden_size=32, and num_layers=2


In [155]:
early_stopper = EarlyStopping(4)

In [156]:
from sklearn.metrics import f1_score

In [157]:
Y_test.float()
for epoch in range(100):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)  # shape: [batch_size]
        loss = criterion(outputs, y_batch.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    model.eval()
    with torch.no_grad():

        for X_batch, y_batch in test_loader:
            test_outputs = model(X_batch)
            test_loss = criterion(test_outputs, y_batch.float())

            test_outputs = test_outputs.round().numpy()
            print(y_batch, "huh", test_outputs)
            f1 = f1_score(y_batch.numpy(), test_outputs)
            acc = (test_outputs == y_batch).float().mean().item()

        # compute F1 (need to convert to numpy)
        
            print(f"Epoch {epoch+1}, "
            f"Val Loss: {test_loss.item():.4f}, "
            f"Val Acc: {acc*100:.2f}%, "
            f"Val F1: {f1:.4f}")
        
    early_stopper(test_loss.item(), model)

    if early_stopper.early_stop:
        print("stopping due to minimal improvements to prevent overfitting")
        break

Epoch 1, Loss: 0.6865
tensor([0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]) huh [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Epoch 1, Val Loss: 0.6397, Val Acc: 87.50%, Val F1: 0.5000
tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 1]) huh [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1.]
Epoch 1, Val Loss: 0.6578, Val Acc: 78.12%, Val F1: 0.2222
tensor([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 1, 0, 0, 1, 0]) huh [1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 1.]
Epoch 1, Val Loss: 0.6949, Val Acc: 50.00%, Val F1: 0.3333
tensor([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 1]) huh [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.

In [167]:
sanity_data_file = "sanity_check.csv"

df = pd.read_csv(data_file)

cols = ["day_of_week", "hour_of_day", "motion_duration"]
X = df[cols].values
y = df["label"].values
scaler = MinMaxScaler()

df[cols] = scaler.fit_transform(df[cols]) # to normalize values since some are orders of magnitude greater than others, e.g. motion_duration

df.head()

finalX = df[cols].values
finalY = df["label"].values

finalX = torch.tensor(finalX)
finalY = torch.tensor(finalY)
final_loader = DataLoader(TensorDataset(finalX, finalY),
                          batch_size=32)

In [171]:
with torch.no_grad():

        for X_batch, y_batch in test_loader:
            test_outputs = model(X_batch)
            test_loss = criterion(test_outputs, y_batch.float())

            test_outputs = test_outputs.round().numpy()
            print(y_batch, "huh", test_outputs)
            f1 = f1_score(y_batch.numpy(), test_outputs)
            acc = (test_outputs == y_batch).float().mean().item()

        # compute F1 (need to convert to numpy)
        
            print(f"Val Loss: {test_loss.item():.4f}, "
            f"Val Acc: {acc*100:.2f}%, "
            f"Val F1: {f1:.4f}")
        

tensor([0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]) huh [0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Val Loss: 0.0283, Val Acc: 100.00%, Val F1: 1.0000
tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 1]) huh [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 0. 0. 1. 1. 0. 0. 1.]
Val Loss: 0.1914, Val Acc: 93.75%, Val F1: 0.8750
tensor([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 1, 0, 0, 1, 0]) huh [0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 1. 1. 1. 0. 1. 0.]
Val Loss: 0.0802, Val Acc: 93.75%, Val F1: 0.9333
tensor([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 1]) huh [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 1. 0

In [172]:
model.load_state_dict(early_stopper.best_state)

<All keys matched successfully>